# Quadrotor (aka "the drone")

## Example of how to set everything up (do this only once)

Import modules and configure the notebook.

In [4]:
import os

# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import matplotlib.pyplot as plt

# This is my own script (it is an interface to the pybullet simulator)
import ae353_drone

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_drone)

<module 'ae353_drone' from '/Users/timothybretl/Documents/courses/AE353/09 - AE353 (Spring 2021)/Website/examples/day40_drone/ae353_drone.py'>

In [5]:
# rng = np.random.default_rng()
rng = np.random.default_rng(0)

In [6]:
rng.standard_normal()

0.1257302210933933

In [7]:
rng.uniform()

0.2697867137638703

In [8]:
for i in range(5):
    print(rng.standard_normal())

0.6404226504432821
0.10490011715303971
-0.535669373161111
0.36159505490948474
1.3040000451301372


Create simulator.

In [10]:
# simulator.disconnect()
simulator = ae353_drone.Simulator(display=False, pos_noise=0., rpy_noise=0., seed=0)

## Example of how to add a drone and run a simulation

Define a controller for the drone.

**NOTE:** This is an example of how to implement three separate PD loops to control roll, pitch, and yaw angles. As we discussed in class, it's unlikely you will take this approach. You will almost certainly choose to implement a controller and observer, for all the reasons we have talked about over the course of the semester. One reason I decided to do an example of PD was because it was an easy thing to design and implement, and so allowed us to look at things like `self.limiter(...)` quickly.

In [11]:
from scipy import linalg

def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

A = np.array([[0., 1.], [0., 0.]])
B = np.array([[0.], [2.]])
C = np.array([[1., 0.]])

# Choose gains
Qc = np.diag([100., 1.])
Rc = np.diag([1.])

# Find optimal gain matrix
K = lqr(A, B, Qc, Rc)

print(f'Gain matrix of controller:\n K = np.array({K.tolist()})\n')

# Find the closed-loop eigenvalues
print(f'Closed-loop eigenvalues of controller:\n {linalg.eigvals(A - B @ K)}')

# Choose gains
Qo = np.diag([1.])
Ro = np.diag([1., 1.])

# Find optimal gain matrix
L = lqr(A.T, C.T, linalg.inv(Ro), linalg.inv(Qo)).T

print(f'Gain matrix of observer:\n L = np.array({L.tolist()})\n')

# Find the closed-loop eigenvalues
print(f'Closed-loop eigenvalues of observer:\n {linalg.eigvals(A - L @ C)}')

Gain matrix of controller:
 K = np.array([[9.999999999999996, 3.316624790355398]])

Closed-loop eigenvalues of controller:
 [-3.31662479+3.j -3.31662479-3.j]
Gain matrix of observer:
 L = np.array([[1.732050807568877], [0.9999999999999993]])

Closed-loop eigenvalues of observer:
 [-0.8660254+0.5j -0.8660254-0.5j]


In [ ]:
class RobotController:
    def __init__(self, limiter=None):
        self.dt = 0.01
        self.limiter = limiter
        
        # Initialize record of old measurements
        self.r_old = 0.
        self.p_old = 0.
        self.y_old = 0.
        
        self.A = np.array([[0., 1.], [0., 0.]])
        self.B = np.array([[0.], [2.]])
        self.C = np.array([[1., 0.]])
        self.K = np.array([[9.999999999999996, 3.316624790355398]])
        self.L = np.array([[1.732050807568877], [0.9999999999999993]])
        
        self.f_z_e = 0.5 * 9.81
        self.z_e = 0.

    def get_color(self):
        return [0., 1., 0.]

    def reset(self, pos):
        self.xhat = np.zeros(2)

    def run(self, pos, rpy, pos_ring, is_last_ring, pos_others):
        
        ##################
        # PD Control
        #
        #  This is easy to implement and allows us to isolate the
        #  "z position and velocity" subsystem, for the purpose of
        #  example.
        #
        
        # Get current measurements of roll, pitch, and yaw
        r = rpy[0]
        p = rpy[1]
        y = rpy[2]
        
        # Estimate roll, pitch, and yaw derivatives by finite difference
        rdot = (r - self.r_old) / self.dt
        pdot = (p - self.p_old) / self.dt
        ydot = (y - self.y_old) / self.dt
        
        # Update record of old measurements
        self.r_old = r
        self.p_old = p
        self.y_old = y
        
        # Choose net torques to drive roll, pitch, and yaw to zero
        tau_x = - 1. * (r - 0) - 0.1 * (rdot - 0)
        tau_y = - 1. * (p - 0) - 0.1 * (pdot - 0)
        tau_z = - 1. * (y - 0) - 0.1 * (ydot - 0)
        
        #
        ##################
        
        zdes = pos_ring[2]
        zest = self.xhat[0] + self.z_e
        max_error = 0.25
        if np.abs(zdes - zest) > max_error:
            zdes = zest + max_error * ((zdes - zest) / linalg.norm(zdes - zest))
        
        
        xdes = np.array([zdes - self.z_e, 0.])
        u = -self.K @ (self.xhat - xdes)
        f_z = u[0] + self.f_z_e
        if self.limiter is not None:
            tau_x, tau_y, tau_z, f_z = self.limiter(tau_x, tau_y, tau_z, f_z)
        u[0] = f_z - self.f_z_e
        
        y = np.array([pos[2] - self.z_e])
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y))
        

        
        
#         print(f'{tau_x}, {tau_y}, {tau_z}, {f_z}')
#         raise Exception('stop')

        return tau_x, tau_y, tau_z, f_z

In [12]:
class RobotController:
    def __init__(self, limiter=None, K=None):
        self.dt = 0.01
        self.limiter = limiter
        
        # Initialize record of old measurements
        self.r_old = 0.
        self.p_old = 0.
        self.y_old = 0.
        
        self.A = np.array([[0., 1.], [0., 0.]])
        self.B = np.array([[0.], [2.]])
        self.C = np.array([[1., 0.]])
        self.K = K
        self.L = np.array([[1.732050807568877], [0.9999999999999993]])
        
        self.f_z_e = 0.5 * 9.81
        self.z_e = 0.

    def get_color(self):
        return [0., 1., 0.]

    def reset(self, pos):
        self.xhat = np.zeros(2)

    def run(self, pos, rpy, pos_ring, is_last_ring, pos_others):
        
        ##################
        # PD Control
        #
        #  This is easy to implement and allows us to isolate the
        #  "z position and velocity" subsystem, for the purpose of
        #  example.
        #
        
        # Get current measurements of roll, pitch, and yaw
        r = rpy[0]
        p = rpy[1]
        y = rpy[2]
        
        # Estimate roll, pitch, and yaw derivatives by finite difference
        rdot = (r - self.r_old) / self.dt
        pdot = (p - self.p_old) / self.dt
        ydot = (y - self.y_old) / self.dt
        
        # Update record of old measurements
        self.r_old = r
        self.p_old = p
        self.y_old = y
        
        # Choose net torques to drive roll, pitch, and yaw to zero
        tau_x = - 1. * (r - 0) - 0.1 * (rdot - 0)
        tau_y = - 1. * (p - 0) - 0.1 * (pdot - 0)
        tau_z = - 1. * (y - 0) - 0.1 * (ydot - 0)
        
        #
        ##################
        
        zdes = pos_ring[2]
        zest = self.xhat[0] + self.z_e
        max_error = 0.25
        if np.abs(zdes - zest) > max_error:
            zdes = zest + max_error * ((zdes - zest) / linalg.norm(zdes - zest))
        
        
        xdes = np.array([zdes - self.z_e, 0.])
        u = -self.K @ (self.xhat - xdes)
        f_z = u[0] + self.f_z_e
        if self.limiter is not None:
            tau_x, tau_y, tau_z, f_z = self.limiter(tau_x, tau_y, tau_z, f_z)
        u[0] = f_z - self.f_z_e
        
        y = np.array([pos[2] - self.z_e])
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y))
        

        
        
#         print(f'{tau_x}, {tau_y}, {tau_z}, {f_z}')
#         raise Exception('stop')

        return tau_x, tau_y, tau_z, f_z

In [13]:
controller = RobotController(K = np.array([[9.999999999999996, 3.316624790355398]]))

Add the drone to the simulator. There are three arguments:

* `RobotController` is the class definition
* `'my_netid'` is the name of the drone
* `'my_image.png'` is the name of an image to use as a label (can be `None`, can be a file path)

You can call this function more than once, so long as you call it with a different drone name each time. An exception will be raised if you try to add two drones of the same name to the simulator.

In [18]:
simulator.clear_drones()
simulator.add_drone(RobotController, 'my_netid', 'my_image.png')
# simulator.load_drones()

Reset the drone (i.e., place it at a random location in the start ring).

In [19]:
simulator.reset(rpy_noise=0.1, linvel_noise=0.1, angvel_noise=0.1)

Run simulation until `max_time` is reached or until the drone reaches the finish ring.

In [20]:
simulator.run(max_time=5.0)


error on run of drone my_netid (turning it off):
Traceback (most recent call last):
  File "/Users/timothybretl/Documents/courses/AE353/09 - AE353 (Spring 2021)/Website/examples/day40_drone/ae353_drone.py", line 514, in step
    tau_x_des, tau_y_des, tau_z_des, f_z_des = drone['controller'].run(pos_meas,
  File "<ipython-input-12-d972d778fad2>", line 67, in run
    u = -self.K @ (self.xhat - xdes)
TypeError: bad operand type for unary -: 'NoneType'



## Example of how to get and plot results

Assume that we already did two things:

* Added a drone with the name `my_netid` to the simulator
* Ran the simulation

Get drone by name.

In [ ]:
drone_name = 'my_netid'
drone = simulator.get_drone_by_name(drone_name)

if drone is None:
    drone_names = '\n'.join([d['name'] for d in simulator.drones])
    msg = f'The simulator has no drone with name "{drone_name}".'
    if len(drone_names) == 0:
        msg += f' The simulator has no drones at all, in fact.'
    else:
        msg += f' The simulator has these drones:'
        msg += f'\n==========\n{drone_names}\n==========\n'
    print(msg)

Extract data.

In [ ]:
data = drone['data'].copy()

Convert all lists in data to numpy arrays.

In [ ]:
for key in data.keys():
    data[key] = np.array(data[key]).T

In [ ]:
data['pos'][0, 0]

Did the drone finish? If so, what was the time at which it finished? If not, was it still running (or was there likely an error)?

In [ ]:
if drone['finish_time'] is None:
    if drone['running']:
        print(f'The drone "{drone["name"]}" did not finish.')
    else:
        print(f'The drone "{drone["name"]}" did not finish and was no longer running.')
else:
    print(f'The drone "{drone["name"]}" finished with time: {drone["finish_time"]}')

Plot results.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_pos, ax_rpy, ax_act) = plt.subplots(3, 1, figsize=(9, 12), sharex=True)

# Position
ax_pos.plot(data['t'], data['pos'][0, :], label='x (m)', linewidth=4)
ax_pos.plot(data['t'], data['pos'][1, :], label='y (m)', linewidth=4)
ax_pos.plot(data['t'], data['pos'][2, :], label='z (m)', linewidth=4)
ax_pos.grid()
ax_pos.legend(fontsize=16)
ax_pos.tick_params(labelsize=14)

# Roll, pitch, and yaw angles
ax_rpy.plot(data['t'], data['rpy'][0, :], label='roll (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][1, :], label='pitch (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][2, :], label='yaw (rad)', linewidth=4)
ax_rpy.grid()
ax_rpy.legend(fontsize=16)
ax_rpy.tick_params(labelsize=14)

# Actuator commands
ax_act.plot(data['t'], data['tau_x'], label='tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_y'], label='tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_z'], label='tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['f_z'], label='f_z (N)', linewidth=4)
ax_act.grid()
ax_act.legend(fontsize=16)
ax_act.tick_params(labelsize=14)

# Set shared x-axis properties
ax_act.set_xlabel('time (s)', fontsize=20)
ax_act.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

Plot error in state estimate

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_z, ax_v) = plt.subplots(2, 1, figsize=(9, 12), sharex=True)

z_tru = data['pos'][2, :]
z_est = data['xhat'][0, :] # <--- assumes equilibrium value of p_z is zero

v_tru = data['linvel'][2, :]
v_est = data['xhat'][1, :] # <--- assumes equilibrium value of v_z is zero

# Position
ax_z.plot(data['t'], z_tru, label='p_z (true)', linewidth=4)
ax_z.plot(data['t'], z_est, '--', label='p_z (estimate)', linewidth=4)
ax_z.grid()
ax_z.legend(fontsize=16)
ax_z.tick_params(labelsize=14)

# Velocity
ax_v.plot(data['t'], v_tru, label='v_z (true)', linewidth=4)
ax_v.plot(data['t'], v_est, '--', label='v_z (estimate)', linewidth=4)
ax_v.grid()
ax_v.legend(fontsize=16)
ax_v.tick_params(labelsize=14)

# Set shared x-axis properties
ax_v.set_xlabel('time (s)', fontsize=20)
ax_v.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

## Example of how to load drones from the "students" directory

Load drones from `students` directory.

In [ ]:
simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to show results, change views, keep running

Show results (so far).

In [ ]:
simulator.show_results()

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Continue running simulation from same point for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

View from finish ring instead of from start ring.

In [ ]:
simulator.camera_finishview()

Continue running simulation from same point for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

Show results (so far).

In [ ]:
simulator.show_results()

## Example of how to clear and reload drones

Clear all drones.

In [ ]:
simulator.clear_drones()

Load drones again from `students` directory.

In [ ]:
simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to rearrange the rings

Rearrange the rings.

In [ ]:
simulator.move_rings()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to take a snapshot

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Display a snapshot.

In [ ]:
# Get snapshot as 480 x 480 x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(9, 9))
plt.imshow(rgba)

Save the snapshot.

In [ ]:
plt.imsave('my_snapshot.png', rgba)

## Example of how to record a movie

You must install both [imageio](https://github.com/imageio/imageio) and [imageio-ffmpeg](https://github.com/imageio/imageio-ffmpeg) in order for this to work. You can do this from a terminal (in your `ae353-bullet` environment, for example) as follows:

```
pip install imageio
pip install imageio-ffmpeg
```

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Run simulation until `max_time` is reached or until all drones finish, saving movie to `my_video.mp4`.

In [ ]:
simulator.run(max_time=1.0, video_filename='my_video.mp4')